<a href="https://colab.research.google.com/github/polyphemus287/IBM-Capstone/blob/main/jupyter_labs_webscraping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Space X  Falcon 9 First Stage Landing Prediction**


## Web scraping Falcon 9 and Falcon Heavy Launches Records from Wikipedia


Estimated time needed: **40** minutes


In this lab, you will be performing web scraping to collect Falcon 9 historical launch records from a Wikipedia page titled `List of Falcon 9 and Falcon Heavy launches`

https://en.wikipedia.org/wiki/List_of_Falcon_9_and_Falcon_Heavy_launches


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/Falcon9_rocket_family.svg)


Falcon 9 first stage will land successfully


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/landing_1.gif)


Several examples of an unsuccessful landing are shown here:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-DS0701EN-SkillsNetwork/api/Images/crash.gif)


More specifically, the launch records are stored in a HTML table shown below:


![](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBM-DS0321EN-SkillsNetwork/labs/module_1_L2/images/falcon9-launches-wiki.png)


  ## Objectives
Web scrap Falcon 9 launch records with `BeautifulSoup`:
- Extract a Falcon 9 launch records HTML table from Wikipedia
- Parse the table and convert it into a Pandas data frame


First let's import required packages for this lab


In [1]:
!pip3 install beautifulsoup4
!pip3 install requests

In [2]:
import sys

import requests
from bs4 import BeautifulSoup
import re
import unicodedata
import pandas as pd

and we will provide some helper functions for you to process web scraped HTML table


In [3]:
def date_time(table_cells):
    """
    This function returns the data and time from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    return [data_time.strip() for data_time in list(table_cells.strings)][0:2]

def booster_version(table_cells):
    """
    This function returns the booster version from the HTML  table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=''.join([booster_version for i,booster_version in enumerate( table_cells.strings) if i%2==0][0:-1])
    return out

def landing_status(table_cells):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    out=[i for i in table_cells.strings][0]
    return out


def get_mass(table_cells):
    mass=unicodedata.normalize("NFKD", table_cells.text).strip()
    if mass:
        mass.find("kg")
        new_mass=mass[0:mass.find("kg")+2]
    else:
        new_mass=0
    return new_mass


def extract_column_from_header(row):
    """
    This function returns the landing status from the HTML table cell
    Input: the  element of a table data cell extracts extra row
    """
    if (row.br):
        row.br.extract()
    if row.a:
        row.a.extract()
    if row.sup:
        row.sup.extract()

    colunm_name = ' '.join(row.contents)

    # Filter the digit and empty names
    if not(colunm_name.strip().isdigit()):
        colunm_name = colunm_name.strip()
        return colunm_name


To keep the lab tasks consistent, you will be asked to scrape the data from a snapshot of the  `List of Falcon 9 and Falcon Heavy launches` Wikipage updated on
`9th June 2021`


In [4]:
static_url = "https://en.wikipedia.org/w/index.php?title=List_of_Falcon_9_and_Falcon_Heavy_launches&oldid=1027686922"

Next, request the HTML page from the above URL and get a `response` object


### TASK 1: Request the Falcon9 Launch Wiki page from its URL


First, let's perform an HTTP GET method to request the Falcon9 Launch HTML page, as an HTTP response.


In [5]:
# use requests.get() method with the provided static_url
# assign the response to a object
# Perform the GET request
response = requests.get(static_url)

# Check if the request was successful
if response.status_code == 200:
    html_content = response.text
    print("Successfully fetched the page!")
else:
    print(f"Failed to fetch the page. Status code: {response.status_code}")

Successfully fetched the page!


Create a `BeautifulSoup` object from the HTML `response`


In [6]:
# Use BeautifulSoup() to create a BeautifulSoup object from a response text content
#`html_content` contains the HTML fetched from the page
soup = BeautifulSoup(response.text, 'html.parser')
# use it to navigate and search through the HTML document
print("BeautifulSoup object has been created.")

BeautifulSoup object has been created.


Print the page title to verify if the `BeautifulSoup` object was created properly


In [7]:
# Use soup.title attribute
page_title = soup.title.text  # This gets the text part of the title tag

print(page_title)

List of Falcon 9 and Falcon Heavy launches - Wikipedia


### TASK 2: Extract all column/variable names from the HTML table header


Next, we want to collect all relevant column names from the HTML table header


Let's try to find all tables on the wiki page first. If you need to refresh your memory about `BeautifulSoup`, please check the external reference link towards the end of this lab


In [8]:
# Use the find_all function in the BeautifulSoup object, with element type `table`
# Assign the result to a list called `html_tables`
# Find all table elements in the soup object
html_tables = soup.find_all('table')

# Display the count of tables found to get an idea of what you're working with
print(f"Total tables found: {len(tables)}")

NameError: name 'tables' is not defined

Starting from the third table is our target table contains the actual launch records.


In [9]:
# Let's print the third table and check its content
first_launch_table = html_tables[2]
print(first_launch_table)

<table class="wikitable plainrowheaders collapsible" style="width: 100%;">
<tbody><tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
<tr>
<th rowspan="2" scope="row" style="text-align:center;">1
</th>
<td>

You should able to see the columns names embedded in the table header elements `<th>` as follows:


```
<tr>
<th scope="col">Flight No.
</th>
<th scope="col">Date and<br/>time (<a href="/wiki/Coordinated_Universal_Time" title="Coordinated Universal Time">UTC</a>)
</th>
<th scope="col"><a href="/wiki/List_of_Falcon_9_first-stage_boosters" title="List of Falcon 9 first-stage boosters">Version,<br/>Booster</a> <sup class="reference" id="cite_ref-booster_11-0"><a href="#cite_note-booster-11">[b]</a></sup>
</th>
<th scope="col">Launch site
</th>
<th scope="col">Payload<sup class="reference" id="cite_ref-Dragon_12-0"><a href="#cite_note-Dragon-12">[c]</a></sup>
</th>
<th scope="col">Payload mass
</th>
<th scope="col">Orbit
</th>
<th scope="col">Customer
</th>
<th scope="col">Launch<br/>outcome
</th>
<th scope="col"><a href="/wiki/Falcon_9_first-stage_landing_tests" title="Falcon 9 first-stage landing tests">Booster<br/>landing</a>
</th></tr>
```


Next, we just need to iterate through the `<th>` elements and apply the provided `extract_column_from_header()` to extract column name one by one


In [10]:
# Apply find_all() function with `th` element on first_launch_table
# Iterate each th element and apply the provided extract_column_from_header() to get a column name
# Append the Non-empty column name (`if name is not None and len(name) > 0`) into a list called column_names
# Initialize an empty list to hold the column names
column_names = []

# Use find_all() to get all <th> elements from the table
th_elements = first_launch_table.find_all('th')

# Iterate over each <th> element
for th in th_elements:
    # Apply the provided extract_column_from_header() function to each <th> element
    name = extract_column_from_header(th)
    # Append the name to column_names list if it is not empty and not None
    if name is not None and len(name) > 0:
        column_names.append(name)

Check the extracted column names


In [11]:
print(column_names)

['Flight No.', 'Date and time ( )', 'Launch site', 'Payload', 'Payload mass', 'Orbit', 'Customer', 'Launch outcome']


## TASK 3: Create a data frame by parsing the launch HTML tables


We will create an empty dictionary with keys from the extracted column names in the previous task. Later, this dictionary will be converted into a Pandas dataframe


In [12]:
launch_dict= dict.fromkeys(column_names)

# Remove an irrelvant column
del launch_dict['Date and time ( )']

# Let's initial the launch_dict with each value to be an empty list
launch_dict['Flight No.'] = []
launch_dict['Launch Site'] = []
launch_dict['Payload'] = []
launch_dict['Payload mass'] = []
launch_dict['Orbit'] = []
launch_dict['Customer'] = []
launch_dict['Launch outcome'] = []
# Added some new columns
launch_dict['Version Booster']=[]
launch_dict['Booster landing']=[]
launch_dict['Date']=[]
launch_dict['Time']=[]

Next, we just need to fill up the `launch_dict` with launch records extracted from table rows.


Usually, HTML tables in Wiki pages are likely to contain unexpected annotations and other types of noises, such as reference links `B0004.1[8]`, missing values `N/A [e]`, inconsistent formatting, etc.


To simplify the parsing process, we have provided an incomplete code snippet below to help you to fill up the `launch_dict`. Please complete the following code snippet with TODOs or you can choose to write your own logic to parse all launch tables:


In [13]:
extracted_row = 0
# Loop through each table found with the specified class to process launch data
for table_number, table in enumerate(soup.find_all('table', "wikitable plainrowheaders collapsible")):
    # Iterate through each row in the current table
    for rows in table.find_all("tr"):
        # Initially, assume the row does not represent a launch entry
        flag = False
        # Check if the row has a table header (th) element and it contains a flight number
        if rows.th:
            if rows.th.string:
                flight_number = rows.th.string.strip()
                # Check if the flight number is purely digits to validate it
                flag = flight_number.isdigit()

        # If there is no valid header, skip to the next iteration
        if not flag:
            continue

        # Now that a valid launch entry is identified, extract all table data (td) elements
        row = rows.find_all('td')

        # Start extracting and appending data to the launch_dict
        extracted_row += 1

        # Extract and print the flight number
        print(f"Flight Number: {flight_number}")
        launch_dict['Flight No.'].append(flight_number)

        # Extract the launch date and time
        datatimelist = date_time(row[0])
        date = datatimelist[0].strip(',')
        time = datatimelist[1]
        print(f"Date: {date}")
        print(f"Time: {time}")
        launch_dict['Date'].append(date)
        launch_dict['Time'].append(time)

        # Extract the booster version
        bv = booster_version(row[1])
        if not(bv):  # Fallback to direct string extraction if the custom function fails
            bv = row[1].a.string if row[1].a else None
        print(f"Version Booster: {bv}")
        launch_dict['Version Booster'].append(bv)

        # Extract and print the launch site
        launch_site = row[2].a.string if row[2].a else None
        print(f"Launch Site: {launch_site}")
        launch_dict['Launch Site'].append(launch_site)

        # Extract and print the payload
        payload = row[3].a.string if row[3].a else None
        print(f"Payload: {payload}")
        launch_dict['Payload'].append(payload)

        # Extract and print the payload mass
        payload_mass = get_mass(row[4])
        print(f"Payload Mass: {payload_mass}")
        launch_dict['Payload mass'].append(payload_mass)

        # Extract and print the orbit
        orbit = row[5].a.string if row[5].a else None
        print(f"Orbit: {orbit}")
        launch_dict['Orbit'].append(orbit)

        # Extract and print the customer
        customer = row[6].a.string if row[6].a else None
        print(f"Customer: {customer}")
        launch_dict['Customer'].append(customer)

        # Extract and print the launch outcome
        launch_outcome = list(row[7].strings)[0].strip()
        print(f"Launch Outcome: {launch_outcome}")
        launch_dict['Launch outcome'].append(launch_outcome)

        # Extract and print the booster landing status
        booster_landing = landing_status(row[8])
        print(f"Booster Landing: {booster_landing}")
        launch_dict['Booster landing'].append(booster_landing)

# At this point, launch_dict is filled with all the data extracted from the HTML tables

Flight Number: 1
Date: 4 June 2010
Time: 18:45
Version Booster: F9 v1.0B0003.1
Launch Site: CCAFS
Payload: Dragon Spacecraft Qualification Unit
Payload Mass: 0
Orbit: LEO
Customer: SpaceX
Launch Outcome: Success
Booster Landing: Failure
Flight Number: 2
Date: 8 December 2010
Time: 15:43
Version Booster: F9 v1.0B0004.1
Launch Site: CCAFS
Payload: Dragon
Payload Mass: 0
Orbit: LEO
Customer: NASA
Launch Outcome: Success
Booster Landing: Failure
Flight Number: 3
Date: 22 May 2012
Time: 07:44
Version Booster: F9 v1.0B0005.1
Launch Site: CCAFS
Payload: Dragon
Payload Mass: 525 kg
Orbit: LEO
Customer: NASA
Launch Outcome: Success
Booster Landing: No attempt

Flight Number: 4
Date: 8 October 2012
Time: 00:35
Version Booster: F9 v1.0B0006.1
Launch Site: CCAFS
Payload: SpaceX CRS-1
Payload Mass: 4,700 kg
Orbit: LEO
Customer: NASA
Launch Outcome: Success
Booster Landing: No attempt
Flight Number: 5
Date: 1 March 2013
Time: 15:10
Version Booster: F9 v1.0B0007.1
Launch Site: CCAFS
Payload: SpaceX C

After you have fill in the parsed launch record values into `launch_dict`, you can create a dataframe from it.


In [14]:
df = pd.DataFrame({key: pd.Series(value, dtype=object) for key, value in launch_dict.items()})

We can now export it to a <b>CSV</b> for the next section, but to make the answers consistent and in case you have difficulties finishing this lab.

Following labs will be using a provided dataset to make each lab independent.


In [15]:
df.to_csv('spacex_web_scraped.csv', index=False)

<code>df.to_csv('spacex_web_scraped.csv', index=False)</code>


## Authors


<a href="https://www.linkedin.com/in/yan-luo-96288783/">Yan Luo</a>


<a href="https://www.linkedin.com/in/nayefaboutayoun/">Nayef Abou Tayoun</a>


## Change Log


| Date (YYYY-MM-DD) | Version | Changed By | Change Description      |
| ----------------- | ------- | ---------- | ----------------------- |
| 2021-06-09        | 1.0     | Yan Luo    | Tasks updates           |
| 2020-11-10        | 1.0     | Nayef      | Created the initial version |


Copyright © 2021 IBM Corporation. All rights reserved.
